In [1]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
df=pd.read_csv(r'NewYork_Times\ArticlesApril2018.csv')
df.head(5)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [3]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [4]:
df['headline'].isnull().values.any()

False

In [5]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [6]:
print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수

총 샘플의 개수 : 1324


In [7]:
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

노이즈값 제거 후 샘플의 개수 : 1214


In [8]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [9]:
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [11]:
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [12]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [13]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [14]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [15]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [16]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [17]:
print(y[:3]) # 레이블

[ 269  371 1115]


In [18]:
y = to_categorical(y, num_classes=vocab_size)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Train on 7803 samples
Epoch 1/200
7803/7803 - 10s - loss: 7.6335 - accuracy: 0.0273
Epoch 2/200
7803/7803 - 3s - loss: 7.1096 - accuracy: 0.0297
Epoch 3/200
7803/7803 - 3s - loss: 6.9712 - accuracy: 0.0343
Epoch 4/200
7803/7803 - 3s - loss: 6.8401 - accuracy: 0.0404
Epoch 5/200
7803/7803 - 3s - loss: 6.6830 - accuracy: 0.0414
Epoch 6/200
7803/7803 - 3s - loss: 6.5080 - accuracy: 0.0478
Epoch 7/200
7803/7803 - 3s - loss: 6.3195 - accuracy: 0.0527
Epoch 8/200
7803/7803 - 3s - loss: 6.1243 - accuracy: 0.0582
Epoch 9/200
7803/7803 - 3s - loss: 5.9311 - accuracy: 0.0631
Epoch 10/200
7803/7803 - 3s - loss: 5.7443 - accuracy: 0.0672
Epoch 11/200
7803/7803 - 3s - loss: 5.5683 - accuracy: 0.0723
Epoch 12/200
7803/7803 - 3s - loss: 5.4015 - accuracy: 0.0814
Epoch 13/200
7803/7803 - 3s - loss: 5.2423 - accuracy: 0.0820
Epoch 14/200
7803/7803 - 3s - loss: 5.0927 - accuracy: 0.0870
Epoch 15/200
7803/7803 - 3s - loss: 4.9484 - accuracy: 0.0992
Epoch 16/200
7803/7803 - 3s - loss: 4.8085 - accuracy: 0

Epoch 133/200
7803/7803 - 3s - loss: 0.3864 - accuracy: 0.9097
Epoch 134/200
7803/7803 - 3s - loss: 0.3921 - accuracy: 0.9099
Epoch 135/200
7803/7803 - 3s - loss: 0.3913 - accuracy: 0.9093
Epoch 136/200
7803/7803 - 3s - loss: 0.3903 - accuracy: 0.9098
Epoch 137/200
7803/7803 - 3s - loss: 0.3622 - accuracy: 0.9123
Epoch 138/200
7803/7803 - 3s - loss: 0.3562 - accuracy: 0.9136
Epoch 139/200
7803/7803 - 3s - loss: 0.3515 - accuracy: 0.9148
Epoch 140/200
7803/7803 - 3s - loss: 0.3475 - accuracy: 0.9141
Epoch 141/200
7803/7803 - 3s - loss: 0.3434 - accuracy: 0.9155
Epoch 142/200
7803/7803 - 3s - loss: 0.3414 - accuracy: 0.9148
Epoch 143/200
7803/7803 - 3s - loss: 0.3377 - accuracy: 0.9154
Epoch 144/200
7803/7803 - 3s - loss: 0.3358 - accuracy: 0.9143
Epoch 145/200
7803/7803 - 3s - loss: 0.3318 - accuracy: 0.9144
Epoch 146/200
7803/7803 - 3s - loss: 0.3280 - accuracy: 0.9153
Epoch 147/200
7803/7803 - 3s - loss: 0.3285 - accuracy: 0.9144
Epoch 148/200
7803/7803 - 3s - loss: 0.3245 - accuracy:

In [21]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [22]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i disapprove of school vouchers can i still apply for them


In [24]:
print(sentence_generation(model, t, 'is', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

is it real policy or is it just reality tv past
